In [1]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
from nba_api.stats.endpoints import playbyplayv2
import time
import random

def is_int_convertible(x):
    try:
        int(x)
        return True
    except (ValueError, TypeError):
        return False


In [25]:
#####data structures to be used

#list of dictionary to append scraped data progressively
df_list=[] #every dict is a game

df= pd.DataFrame(columns=['GAME_ID',
                          'TEAM1',
                          'TEAM2', 'DATE', 'PF', '3PM','DUNKS', 'OT', 'CLUTCH'])

In [2]:
#####game all games id

# get game logs from the reg season
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2024-25',
                                              league_id_nullable='00',
                                              season_type_nullable='Regular Season')
games = gamefinder.get_data_frames()[0]

# get a list of the distinct game_ids
game_ids = games['GAME_ID'].unique().tolist()
len(game_ids)

1230

In [19]:
######get data for each game of the season
for game in game_ids:
    dict={}

#add game id
    dict['GAME_ID']=game

#get teams, date,  fouls, 3pm
    dict['TEAM1']= str(games[games["GAME_ID"]== game].iloc[0]["TEAM_ABBREVIATION"])
    dict['TEAM2']= str(games[games["GAME_ID"]== game].iloc[1]["TEAM_ABBREVIATION"])
    dict['DATE']= str(games[games["GAME_ID"]== game].iloc[0]["GAME_DATE"])
    dict['PF']= games[games["GAME_ID"]== game].iloc[0]["PF"] + games[games["GAME_ID"]== game].iloc[1]["PF"]
    dict['3PM']= games[games["GAME_ID"]== game].iloc[0]["FG3M"] + games[games["GAME_ID"]== game].iloc[1]["FG3M"]

#extract 'dunks', 'OT' using period, 'clutch' using time and score margin
    pbp = playbyplayv2.PlayByPlayV2(game)
    pbp = pbp.get_data_frames()[0]

    dict['DUNKS']= pbp[pbp["DESCRIPTION"].str.contains("Dunk", case=False, na=False) & ~pbp["HOMEDESCRIPTION"].str.contains("MISS", case=False, na=False) |
                        pbp["VISITORDESCRIPTION"].str.contains("Dunk", case=False, na=False) & ~pbp["VISITORDESCRIPTION"].str.contains("MISS", case=False, na=False) ].shape[0]

    OT= pbp['PERIOD'].unique().max()
    if OT==5:
        OT=1
    elif OT==6:
        OT=2
    elif OT==7:
        OT=3
    else:
        OT=0
    dict['OT']= OT

    pbp_valid= pbp[(pbp["PERIOD"]>=4) & (pbp["PCTIMESTRING"].str.split(":").str[0].astype(int) < 5) & (pbp['SCOREMARGIN'].notna()) & (pbp['SCOREMARGIN'].apply(is_int_convertible))]
    clutch=(pbp_valid["SCOREMARGIN"].astype(int) <= 5) & (pbp_valid["SCOREMARGIN"].astype(int) >= -5)
    dict['CLUTCH']= 1 if True in clutch.unique() else 0

    df_list.append(dict)

            # Respectful delay to avoid rate limiting
    time.sleep(random.uniform(0.8, 1.5))

len(df_list)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [14]:
dftemp = pd.DataFrame(df_list, columns=['GAME_ID', 'TEAM1', 'TEAM2', 'DATE', 'PF', '3PM', 'DUNKS', 'OT', 'CLUTCH'])
dftemp

,GAME_ID,TEAM1,TEAM2,DATE,PF,3PM,DUNKS,OT,CLUTCH
0,0022401186,ATL,ORL,2025-04-13,23,32,16,0,0
1,0022401188,BKN,NYK,2025-04-13,49,28,5,0,1
2,0022401195,UTA,MIN,2025-04-13,31,29,11,0,0
3,0022401194,MEM,DAL,2025-04-13,28,20,19,0,0
4,0022401196,OKC,NOP,2025-04-13,44,16,6,0,0
5,0022401200,SAC,PHX,2025-04-13,16,29,7,0,0
6,0022401192,MIL,DET,2025-04-13,36,45,5,1,1
7,0022401187,BOS,CHA,2025-04-13,30,25,4,0,1
8,0022401199,POR,LAL,2025-04-13,37,24,12,0,0
9,0022401193,DEN,HOU,2025-04-13,37,22,13,0,0


In [7]:
game_ids[36:40]

['0022401164', '0022401157', '0022401156', '0022401160']